In [1]:
!pip install langchain openai chromadb tiktoken jq


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 15.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 13.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 13.6 MB/s eta 0:00:0000:0100:01
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.7 MB/s eta 0:00:00
  Created wheel for pypika: filename=pypika-0.48.9-py2.py3-none-any.whl size=53803 sha256=8d78386a6c07de43b73fccf9c018c7da9e4a51beeab70bb96ca22d9f1727bde2
  St

In [1]:
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import JSONLoader

In [2]:
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["episode"] = record.get("episode")
    metadata["role"] = record.get("role")
    metadata["speaker"] = record.get("speaker")
    return metadata


In [3]:
loader = JSONLoader(
    file_path="data/transcripts_full.json",
    jq_schema=".[].[]",     
    content_key="utterance",  
    metadata_func=metadata_func
)

docs = loader.load()

In [4]:
print(docs[0]
)

page_content='Joe Franklin?' metadata={'source': '/Users/matthewlu/Downloads/podcast-RAG/data/transcripts_full.json', 'seq_num': 1, 'episode': 'ep-1', 'role': 'interviewer', 'speaker': 'ira glass'}
